In [1]:
from __future__ import print_function
from collections import defaultdict
import re
import json
import os
import io

In [2]:
quotes = []
movies = []
year = []
movies_to_quotes = defaultdict(list)

In [3]:
# 400 quote data from http://www.afi.com/Docs/100Years/quotes400.pdf
# read file
path = 'movie_quotes/'
file0 = path+'raw_movie_quotes_afi400.txt'
chunks = ''
with open(file0) as fin:
    data = fin.read()
    chunks = re.findall('(?:(?!\n[0-9])[^<>])*',data)
    
while '' in chunks: # filter out empty texts
    chunks.remove('')

# split on new lines (\n)
lines = []
for line in chunks: 
    lines.append(line.split("\n"))

# remomve sayer, production, etc.
new_lines = []
for line in lines:
    length = len(line)
    cleaned_line = []
    for item in line[1:]:
        if not (item.startswith("ACTOR ") or item.startswith("ACTORS ") or \
                item.startswith("DIRECTOR ") or item.startswith("DIRECTORS ") or \
                item.startswith("SCREENWRITER ") or item.startswith("SCREENWRITERS ") or \
                item.startswith("PRODUCER ") or item.startswith("PRODUCERS ") or \
                item.startswith("AFI is a trademark of the American Film Institute")):
            cleaned_line.append(item)
    new_lines.append(cleaned_line)

while '' in new_lines: # filter out empty texts
    new_lines.remove('')
    
movie_quotes =[]
for line in new_lines:
    if line[0].startswith("Dave, stop."):
        continue
    length = len(line)
    if length >= 3:
        if ((len(line[length-1]) == 0) or line[length-1].startswith("Merrill De Maris, Dorothy Ann")):
            company = line[length-2]
            movie = line[length-3]
            quote = ""
            for s in range(length-3):
                quote = quote+line[s]+" "
            movie_quotes.append([quote,movie,company])
        else:
            company = line[length-1]
            movie = line[length-2]
            quote = ""
            for s in range(length-2):
                quote = quote+line[s]+" "
            movie_quotes.append([quote,movie,company])
    else:
        print('ERROR reading file0')

#manual fixing of 3 cases
movie_quotes.append(["I'm your worst fucking nightmare, man! A nigger with a badge.", "48 HOURS", "Paramount, 1982"])
movie_quotes.append(["Sawyer, you're going out a youngster, but you've got to come back a star!", "42ND STREET", "Warner Bros., 1933"])
movie_quotes.append(["Dave, stop. Stop, will you Stop, Dave. Will you stop, Dave Stop, Dave. I'm afraid.", "2001 A SPACE ODYSSEY", "MGM, 1968"])
movie_quotes.append(["Open the pod doors, HAL.","2001 A SPACE ODYSSEY", "MGM, 1968"])

def split_movie_lines(line): #consider each line in multi speaker movie quotes its own quote
    words = line.split()
    fixed = []
    line = ""
    for x in range(len(words)):
        if (words[x].isupper()) and not(words[x] == "I"):
            if (len(line.strip()) > 0):
                fixed.append(line.strip())
            line = ""
        else:
            line = line + " " + words[x]
    if (len(line.strip()) > 0):
        fixed.append(line.strip())
    return fixed

for line in movie_quotes:
    if (len(line) == 3):
        q = split_movie_lines(line[0].replace('\n',''))
        m = line[1].replace('\n','')
        company = line[2].replace('\n','')
        date = company.split(', ')
        if (len(date) == 2):
            y = date[1]
        else:
            print("error finding year")
            y = "???"
            print(line)
        for x in q:
            quotes.append(x.lower().strip())
            movies.append(m.lower().strip())
            year.append(y)
            movies_to_quotes[m.lower().strip()].append(x)
    else:
        print(line)

print(len(set(movies)))
print(len(set(quotes)))
print(len(set(year)))

ERROR reading file0
ERROR reading file0
ERROR reading file0
ERROR reading file0
ERROR reading file0
ERROR reading file0
ERROR reading file0
341
459
74


In [4]:
# quote data from http://www.afi.com
file1 = path+'raw_movie_quotes_afibest1990.txt'
file2 = path+'raw_movie_quotes_afibest2000.txt'
file3 = path+'raw_movie_quotes_afifunny.txt'

quote_by_quote = []
# read files and amass all quotes
with open(file1) as fin:
    data = fin.readlines()
    quote = []
    for line in data:
        if (len(line.strip()) == 0):
            quote_by_quote.append(quote)
            quote = []
        else:
            quote.append(line.replace('\n','').replace('"',''))
    quote_by_quote.append(quote)
with open(file2) as fin:
    data = fin.readlines()
    quote = []
    for line in data:
        if (len(line.strip()) == 0):
            quote_by_quote.append(quote)
            quote = []
        else:
            quote.append(line.replace('\n','').replace('"',''))
    quote_by_quote.append(quote)
with open(file3) as fin:
    data = fin.readlines()
    quote = []
    for line in data:
        if (len(line.strip()) == 0):
            quote_by_quote.append(quote)
            quote = []
        else:
            quote.append(line.replace('\n','').replace('"',''))
    quote_by_quote.append(quote)

#process quotes
for quote_lines in quote_by_quote:
    for line in quote_lines[:(len(quote_lines)-2)]:
        stripped = line.strip()
        clean = stripped.replace("- ", '').lower()
        if clean not in quotes:
            almost = quote_lines[len(quote_lines)-2]
            y = almost[almost.find("(")+1:almost.find(")")].strip()
            m = re.sub(r'\([^)]*\)', '', almost).strip().lower()
            q = clean
            quotes.append(q)
            movies.append(m)
            year.append(y)
            movies_to_quotes[m].append(q)

print(len(set(movies)))
print(len(set(quotes)))
print(len(set(year)))

480
1075
86


In [5]:
# quote data from http://www.afi.com
file4 = path+'raw_movie_quotes_afi50great.txt'
file5 = path+'raw_movie_quotes_afipopular.txt'

def afi_movie_after_speaker(line):
    tokens = line.lower().split()
    counter = 0
    for tk in tokens:
        if tk == "in":
            counter = counter + 3
            break
        else:
            counter = counter + len(tk) + 1
    return line[counter:]


quote_by_quote = []
# read files and amass all quotes
with open(file4) as fin:
    data = fin.readlines()
    quote = []
    for line in data:
        if (len(line.strip()) == 0):
            quote_by_quote.append(quote)
            quote = []
        else:
            quote.append(line.replace('\n','').replace('"',''))
    quote_by_quote.append(quote)
with open(file5) as fin:
    data = fin.readlines()
    quote = []
    for line in data:
        if (len(line.strip()) == 0):
            quote_by_quote.append(quote)
            quote = []
        else:
            quote.append(line.replace('\n','').replace('"',''))
    quote_by_quote.append(quote)

for quote_lines in quote_by_quote:
    for line in quote_lines[:(len(quote_lines)-2)]:
        stripped = line.strip()
        clean = stripped.replace("- ", '').lower()
        if clean not in quotes:
            speaker_junk = quote_lines[len(quote_lines)-2]
            almost = afi_movie_after_speaker(speaker_junk)
            y = almost[almost.find("(")+1:almost.find(")")].strip()
            m = re.sub(r'\([^)]*\)', '', almost).strip().lower()
            q = clean
            quotes.append(q)
            movies.append(m)
            year.append(y)
            movies_to_quotes[m].append(q)

print(len(set(movies)))
print(len(set(quotes)))
print(len(set(year)))

490
1155
87


In [6]:
with io.open("movies.json",'w',encoding="utf-8") as fout:
  fout.write(unicode(json.dumps(movies, ensure_ascii=False)))
with io.open("quotes.json",'w',encoding="utf-8") as fout:
  fout.write(unicode(json.dumps(quotes, ensure_ascii=False)))
with io.open("year.json",'w',encoding="utf-8") as fout:
  fout.write(unicode(json.dumps(year, ensure_ascii=False)))
with io.open("movies_to_quotes.json",'w',encoding="utf-8") as fout:
  fout.write(unicode(json.dumps(movies_to_quotes, ensure_ascii=False)))

['all-righty then!', 'elementary, my dear watson.', "oh, it was nobody's fault but my own. i was looking up. it was the nearest thing to heaven. you were there.", "man takes a drop too much once in a while, it's only human nature.", '"nature," mr. allnut, is what we are put into this world to rise above.', "surely you can't be serious.", "i am serious. and don't call me shirley.", 'looks like i picked the wrong week to stop sniffing glue.', 'you ever been in a cockpit before?', "no sir, i've never been up in a plane before.", 'you ever seen a grown man naked?', 'get away from her, you bitch!', "fasten your seatbelts. it's going to be a bumpy night.", 'and our bodies are earth. and our thoughts are clay. and we sleep and eat with death.', "it's showtime!", 'follow the money.', 'there are simply too many notes.', "sometimes there's so much beauty in the world i feel like i can't take it, like my heart's going to cave in.", "one morning i shot an elephant in my pajamas. how he got in my p